Copyright 2020-2022 Universidad Complutense de Madrid

This file software has been employed to calibrate optical spectroscopic data from the OSIRIS instrument at GTC (see [Paliya et al. 2020](https://ui.adsabs.harvard.edu/abs/2020ApJ...903L...8P/abstract))

Authors: Cristina Cabello (criscabe@ucm.es), Nicolás Cardiel (cardiel@ucm.es) Jesús Gallego (j.gallego@ucm.es)

SPDX-License-Identifier: GPL-3.0+ License-Filename: LICENSE.txt

In [1]:
import matplotlib.pyplot as plt

from astropy.io import fits
import numpy as np

# <span style="color:red"> Reduction of the OSIRIS target spectrum</span>  

# 1) Observing Block 2D images: image registration

<span style="color:darkblue"> We had a look at the object images of each Observing Block (OB):</span>



```
$ cd OB0001/object

$ fitsheader *.fits -k object -k exptime -f
                         filename                          OBJECT EXPTIME
---------------------------------------------------------- ------ -------
0002581310-20200526-OSIRIS-OsirisLongSlitSpectroscopy.fits  J1219    30.0
0002581311-20200526-OSIRIS-OsirisLongSlitSpectroscopy.fits  J1219    30.0
0002581312-20200526-OSIRIS-OsirisLongSlitSpectroscopy.fits  J1219    30.0
0002581313-20200526-OSIRIS-OsirisLongSlitSpectroscopy.fits  J1219    30.0
0002581314-20200526-OSIRIS-OsirisLongSlitSpectroscopy.fits  J1219    30.0
0002581315-20200526-OSIRIS-OsirisLongSlitSpectroscopy.fits  J1219  1200.0
0002581316-20200526-OSIRIS-OsirisLongSlitSpectroscopy.fits  J1219  1200.0
0002581317-20200526-OSIRIS-OsirisLongSlitSpectroscopy.fits  J1219  1200.0

```


```
$ cd OB0002/object

$ fitsheader *.fits -k object -k exptime -f
                         filename                          OBJECT EXPTIME
---------------------------------------------------------- ------ -------
0002581318-20200526-OSIRIS-OsirisLongSlitSpectroscopy.fits  J1219    30.0
0002581319-20200526-OSIRIS-OsirisLongSlitSpectroscopy.fits  J1219    30.0
0002581320-20200526-OSIRIS-OsirisLongSlitSpectroscopy.fits  J1219    30.0
0002581321-20200526-OSIRIS-OsirisLongSlitSpectroscopy.fits  J1219    30.0
0002581322-20200526-OSIRIS-OsirisLongSlitSpectroscopy.fits  J1219  1200.0
0002581323-20200526-OSIRIS-OsirisLongSlitSpectroscopy.fits  J1219  1200.0
0002581324-20200526-OSIRIS-OsirisLongSlitSpectroscopy.fits  J1219  1200.0
```

<span style="color:darkblue"> We have 3 exposures of 1200 seconds in each OB. The individual science images were bias subtracted, trimmed, rotated and divided by the MasterFlat. </span>


In [ ]:
# read median bias frame

with fits.open('../bias/bias_median.fits') as hdul:
    header = hdul[0].header
    bias = hdul[0].data



# read trimmed and rotated normalized flat

with fits.open('../flat/flat_norm.fits') as hdul:
    header = hdul[0].header
    flat_norm = hdul[0].data

    
#### OB0001
    
for i in range(15, 18):
    fname = '00025813{}-20200526-OSIRIS-OsirisLongSlitSpectroscopy.fits'.format(i)
    with fits.open(fname) as hdul:
        header = hdul[2].header
        data = hdul[2].data
        
    # subtract bias
    data = data - bias
    
    # trim image
    data = data[:, 25:950]
    
    # rotate image
    data = np.rot90(data, 1)
    
    # divide by flatfield
    data = data / flat_norm

    # save image
    hdu = fits.PrimaryHDU(data, header=header)
    hdulist = fits.HDUList(hdu)
    outfile = '{}brf.fits'.format(i)
    hdulist.writeto(outfile, overwrite=True)   
    
    
#### OB0002

for i in range(22, 25):
    fname = '00025813{}-20200526-OSIRIS-OsirisLongSlitSpectroscopy.fits'.format(i)
    with fits.open(fname) as hdul:
        header = hdul[2].header
        data = hdul[2].data
        
    # subtract bias
    data = data - bias
    
    # trim image
    data = data[:, 25:950]
    
    # rotate image
    data = np.rot90(data, 1)
    
    # divide by flatfield
    data = data / flat_norm

    # save image
    hdu = fits.PrimaryHDU(data, header=header)
    hdulist = fits.HDUList(hdu)
    outfile = '{}brf.fits'.format(i)
    hdulist.writeto(outfile, overwrite=True) 

<span style="color:darkblue"> Furthermore, we used the cleanest (https://cleanest.readthedocs.io/en/latest/) software [(Cardiel 2020)](https://ui.adsabs.harvard.edu/abs/2020ASPC..522..723C/abstract) to interpolate the defective pixels in the individual images.</span>

<span style="color:darkblue"> The three individual images of each OB were then combined using the mean. Thus, the resulting combined images of the OB1 and OB2 had the signal corresponded to an individual exposure of 1200s.</span>

<span style="color:darkblue">For the OB0001:</span>
```
$ ls 1?brfc.fits > list_brfc.txt

$ numina-r6-imcombine list_brfc.txt --method mean ob1.fits
<--15brfc.fits (image 1 of 3)
<--16brfc.fits (image 2 of 3)
<--17brfc.fits (image 3 of 3)
==> Generating output file: ob1.fits...
```

<span style="color:darkblue">For the OB0002:</span>

```
$ ls 2?brfc.fits > list_brfc.txt

$ numina-r6-imcombine list_brfc.txt --method mean ob2.fits
<--22brfc.fits (image 1 of 3)
<--23brfc.fits (image 2 of 3)
<--24brfc.fits (image 3 of 3)
==> Generating output file: ob2.fits...
```

# 2) Sky subtraction


<span style="color:darkblue"> Then we carried out the sky subtraction using the xnirspec (https://github.com/nicocardiel/xnirspec) software [(Cardiel et al. 2003)](https://ui.adsabs.harvard.edu/abs/2003ApJ...584...76C/abstract). </span>
    
<span style="color:darkblue">Since the target was placed in different positions in the two OBs, we removed the major contribution of skylines by subtracting these two images (first order sky subtraction). </span>

<span style="color:darkblue">Second order subtraction: ... </span>

# 3) REDUCEME

<span style="color:darkblue"> We used the ReDucEme (https://reduceme.readthedocs.io/en/latest/) reduction package [(Cardiel 1999)](https://ui.adsabs.harvard.edu/abs/1999PhDT........12C/abstract) for the further reduction steps.</span>


<span style="color:darkblue"> First, we converted the resulting images from FITS format to REDUCEME format. </span>

```
$ R5-leefits_new: b1_2.fits -> b1_2.u

$ R5-leefits_new: b2_1.fits -> b2_1.u
```


### - 3.1) C-distortion correction and wavelength calibration

<span style="color:darkblue"> We applied the C-distortion correction and the wavelength calibration previously computed (see `00_calibrations.ipynb`) to the sky-subtracted images using the ‘rebincw’ function:</span>

```
$ R5-rebincw 
*******************************************************************************
REDUCEMEv5.0                  Welcome to rebincw       Version: 8-December-1996
-------------------------------------------------------------------------------


-------------------------------------------------------------------------------
                               Program REBINCW
    This program needs data (files) generated through FITCDIS and FITLIN.
    Since these files are suposed to be in a directory containing all the
    files from arc reduction, REBINCW will search for the output files of
    FITCIDS and FITLIN  in the current directory first, and,  if they are
    not present there, the search will continue in the directory ../cuar/.
-------------------------------------------------------------------------------

Work with error images (y/n) [n] ? 
Input file name? b1_2.u
>>> NSCAN :    925
>>> NCHAN :   2051
>>> STWV  : 0.00000000
>>> DISP  : 0.00000000
>>> OBJECT: [not found]

Correct from C-distortion (y/n) [y] ? 
Number of arcs to be used  (1,...,2) [1] ? 

ARC #     1

>>> C-DISTORTION CORRECTION:
Spectra for which the corr. was derived: from... to... [1,925] ? 
Order of polynomial  (0,...,19)? 2
How many polynomial to be applied (1=no iteration) [1] ? 
Iteration # 1: Polynomial file name (from fitcdis)? ../OB0001/arc/arc.cdis1

>>> WAVELENGTH CALIBRATION:
Polynomial file name (from fitlin)? ../OB0001/arc/polarc
>>> Polynomial degree:            7


Enter STWV (central wavelenght of pixel 1)? 3644.58765
Enter desired dispersion (angstroms/pixel)? 2.06242371

Pixel     1    -------> Blank from scan     1 to     1
Pixel     2    -------> Blank from scan     1 to     1
Pixel     3    -------> Blank from scan     1 to     1
Pixel     4    -------> Blank from scan     1 to     1
Pixel     5    -------> Blank from scan     1 to     1
Pixel     6    -------> Blank from scan     1 to     1
Pixel     7    -------> Blank from scan     1 to     1
Pixel     8    -------> Blank from scan     1 to     1
Pixel     9    -------> Blank from scan     1 to     1
Pixel    10    -------> Blank from scan     1 to     1
Pixel    11    -------> Blank from scan     1 to     1
Pixel    12    -------> Blank from scan     1 to     1
Pixel    13    -------> Blank from scan     1 to     1
Pixel    14    -------> Blank from scan     1 to     1
Pixel    15    -------> Blank from scan     1 to     1
Pixel    16    -------> Blank from scan     1 to     1
Pixel    17    -------> Blank from scan     1 to     1
Pixel    18    -------> Blank from scan     1 to     1

>>> Checking conservation of number of counts...   ...OK!
>>> Maximum error (%):    4627.98779    
    in scan no.:    471

Shift spectra (radial velocitiy) (y/n) [n] ? 

Output file name? b1_2.uw

```

<span style="color:darkblue"> We followed the same procedure to generate the `b2_1.uw` image.</span>

### - 3.2) S-distortion correction

<span style="color:darkblue"> We used the ‘sdistor’ function to rectify the shape of the spectra by correcting for distortion S. </span>
 
<span style="color:darkblue">For the OB0001:</span>


```
$ R5-sdistor 
*******************************************************************************
REDUCEMEv5.0                  Welcome to sdistor         Version: 10-March-2005
-------------------------------------------------------------------------------

Work with error images (y/n) [y] ? 
Graphic device #1 (? to see list) [/XServe] ? 
Graphic device #2 (NONE=EXIT) (? to see list) [NONE] ? 
Input file name......? b1_2.uw
>>> NSCAN :    925
>>> NCHAN :   2051
>>> STWV  : 3644.58765
>>> DISP  : 2.06242371
>>> OBJECT: [not found]
Input error file name [b1_2e.uw] ? 
>>> NSCAN :    925
>>> NCHAN :   2051
>>> STWV  : 3644.58765
>>> DISP  : 2.06242371
>>> OBJECT: [not found] @ERROR@
Perform fit to:
    1 : Simmetric central region
    2 : More general region
    3 : Use user-defined polynomial for distortion
Option (1/2/3) [1] ? 
Channel region to plot averaged spatial profile [1,2051] ? 
>>> Maximum is located at scan:    702
Search for a different maximum (y/n) [n] ? y
Define scan region: 
1st scan  (1,...,925)? 517
2nd scan  (517,...,925)? 528
>>> Maximum is located at scan:    524
Search for a different maximum (y/n) [n] ? 
Center scan  (1,...,925) [524] ? 
No. of scans at each side to find/fit maximum [3] ? 
Enter channels to be fitted (0,0=EXIT):
Channel region [0,0] ? 200,2051
Channel region [0,0] ? 
Binning (y/n) [n] ? y
Bin width[1] ? 50
No. of scans around center scan to be plotted  (1,...,925) [4] ? 5
Plot fits to Cauchy functions for each channel (y/n) [n] ? 
>>> Original center scan:          524
>>> Fitted center scan..:          523
Working...
Press RETURN to continue...
Mean deviation around central scan (scans):   0.683060467    
Polynomial degree (<20,-2=RESTART,-3=QUIT)? 2
No. of SCANS from center scan to exclude points [3] ? 
No. of points too far from the center scan:      0
Times SIGMA to exclude points [3.0] ? 
No. of points removed from fit:      0
Coefficients from fit:
> a(0) =    519.959167    
> a(1) =    6.81946520E-03
> a(2) =   -2.92290315E-06
Mean dispersion around polynomial  :    7.36610815E-02

Change polynomial degree (y/n) [n] ? 
Working...
Press RETURN to continue...
Mean deviation around central scan (scans):    8.54331553E-02
Polynomial degree (<20,-1=NO MORE FITS,-3=QUIT)? -1
Save input file after initial correction (y/n) [n] ? 

* Plotting fits to local polynomials:
Channel (0=EXIT) (0,...,2051)? 0
No. of iterations to link local fits  (0,...,100) [0] ? 
Working with initial frame...
channell:    100...
channell:    200...
channell:    300...
channell:    400...
channell:    500...
channell:    600...
channell:    700...
channell:    800...
channell:    900...
channell:   1000...
channell:   1100...
channell:   1200...
channell:   1300...
channell:   1400...
channell:   1500...
channell:   1600...
channell:   1700...
channell:   1800...
channell:   1900...
channell:   2000...   ...OK!
Working with error frame...
channell:    100...
channell:    200...
channell:    300...
channell:    400...
channell:    500...
channell:    600...
channell:    700...
channell:    800...
channell:    900...
channell:   1000...
channell:   1100...
channell:   1200...
channell:   1300...
channell:   1400...
channell:   1500...
channell:   1600...
channell:   1700...
channell:   1800...
channell:   1900...
channell:   2000...   ...OK!
Output file name.......(corrected from S-distortion)? b1_2.uwss
Output error file name (corrected from S-distortion) [b1_2e.uwss] ? 
```

<span style="color:darkblue">For the OB0002:</span>


```

$ R5-sdistor 
*******************************************************************************
REDUCEMEv5.0                  Welcome to sdistor         Version: 10-March-2005
-------------------------------------------------------------------------------

Work with error images (y/n) [y] ? 
Graphic device #1 (? to see list) [/XServe] ? 
Graphic device #2 (NONE=EXIT) (? to see list) [NONE] ? 
Input file name......? b2_1.uw
>>> NSCAN :    925
>>> NCHAN :   2051
>>> STWV  : 3644.58765
>>> DISP  : 2.06242371
>>> OBJECT: [not found]
Input error file name [b2_1e.uw] ? 
>>> NSCAN :    925
>>> NCHAN :   2051
>>> STWV  : 3644.58765
>>> DISP  : 2.06242371
>>> OBJECT: [not found] @ERROR@
Perform fit to:
    1 : Simmetric central region
    2 : More general region
    3 : Use user-defined polynomial for distortion
Option (1/2/3) [1] ? 
Channel region to plot averaged spatial profile [1,2051] ? 
>>> Maximum is located at scan:    752
Search for a different maximum (y/n) [n] ? y
Define scan region: 
1st scan  (1,...,925)? 570
2nd scan  (570,...,925)? 578
>>> Maximum is located at scan:    574
Search for a different maximum (y/n) [n] ? 
Center scan  (1,...,925) [574] ? 
No. of scans at each side to find/fit maximum [3] ? 
Enter channels to be fitted (0,0=EXIT):
Channel region [0,0] ? 200,2051
Channel region [0,0] ? 
Binning (y/n) [n] ? y
Bin width[1] ? 50
No. of scans around center scan to be plotted  (1,...,925) [4] ? 5
Plot fits to Cauchy functions for each channel (y/n) [n] ? 
>>> Original center scan:          574
>>> Fitted center scan..:          574
Working...
Press RETURN to continue...
Mean deviation around central scan (scans):   0.536545575    
Polynomial degree (<20,-2=RESTART,-3=QUIT)? 2
No. of SCANS from center scan to exclude points [3] ? 
No. of points too far from the center scan:      0
Times SIGMA to exclude points [3.0] ? 
No. of points removed from fit:      2
Coefficients from fit:
> a(0) =    571.372009    
> a(1) =    5.43402787E-03
> a(2) =   -2.39725864E-06
Mean dispersion around polynomial  :   0.226018712    

Change polynomial degree (y/n) [n] ? 
Working...
Press RETURN to continue...
Mean deviation around central scan (scans):   0.104159705    
Polynomial degree (<20,-1=NO MORE FITS,-3=QUIT)? -1
Save input file after initial correction (y/n) [n] ? 

* Plotting fits to local polynomials:
Channel (0=EXIT) (0,...,2051)? 0
No. of iterations to link local fits  (0,...,100) [0] ? 
Working with initial frame...
channell:    100...
channell:    200...
channell:    300...
channell:    400...
channell:    500...
channell:    600...
channell:    700...
channell:    800...
channell:    900...
channell:   1000...
channell:   1100...
channell:   1200...
channell:   1300...
channell:   1400...
channell:   1500...
channell:   1600...
channell:   1700...
channell:   1800...
channell:   1900...
channell:   2000...   ...OK!
Working with error frame...
channell:    100...
channell:    200...
channell:    300...
channell:    400...
channell:    500...
channell:    600...
channell:    700...
channell:    800...
channell:    900...
channell:   1000...
channell:   1100...
channell:   1200...
channell:   1300...
channell:   1400...
channell:   1500...
channell:   1600...
channell:   1700...
channell:   1800...
channell:   1900...
channell:   2000...   ...OK!
Output file name.......(corrected from S-distortion)? b2_1.uwss
Output error file name (corrected from S-distortion) [b2_1e.uwss] ? 


```

<img src="../plots_corrections/plots_finales/00_S_distortion.png" width="600px">


### - 3.3) Atmospheric extinction correction

<span style="color:darkblue"> Finally, we corrected for atmospheric extinction with the ‘corrext’ function, taking into account that the airmass was 1.06 for the OB1 observations and 1.20 for those of the OB2.</span>

<span style="color:darkblue">For the OB0001:</span>


```
$ R5-corrext 
*******************************************************************************
REDUCEMEv5.0                  Welcome to corrext       Version: 6-December-1996
-------------------------------------------------------------------------------

Work with error images (y/n) [n] ? y
Input file name? b1_2.uwss
>>> NSCAN :    925
>>> NCHAN :   2051
>>> STWV  : 3644.58765
>>> DISP  : 2.06242371
>>> OBJECT: [not found]
Input error file name [b1_2e.uwss] ? 
>>> NSCAN :    925
>>> NCHAN :   2051
>>> STWV  : 3644.58765
>>> DISP  : 2.06242371
>>> OBJECT: [not found] @ERROR@
STWV = 3644.58765765                                     
DISP = 2.06242371371                                     
Are these values OK (y/n) [y] ? 
Correct from atmospheric extinction (y/n) [y] ? 
1 - Curve from file
2 - Semi-analytical function
Option (1/2) [2] ? 1
Extinction curve file name [/files/extlp.dat] ? extlp.dat
No. points read:    201
Mean air mass [-999.000000] ? 1.06
Interpolating function...   ...OK
Correct from interstellar extinction (y/n) [y] ? n
Output file name? b1_2.uwssa
Output error file name [b1_2e.uwssa] ? 
```

<span style="color:darkblue">For the OB0002:</span>


```
$ R5-corrext 
*******************************************************************************
REDUCEMEv5.0                  Welcome to corrext       Version: 6-December-1996
-------------------------------------------------------------------------------

Work with error images (y/n) [n] ? y
Input file name? b2_1.uwss
>>> NSCAN :    925
>>> NCHAN :   2051
>>> STWV  : 3644.58765
>>> DISP  : 2.06242371
>>> OBJECT: [not found]
Input error file name [b2_1e.uwss] ? 
>>> NSCAN :    925
>>> NCHAN :   2051
>>> STWV  : 3644.58765
>>> DISP  : 2.06242371
>>> OBJECT: [not found] @ERROR@
STWV = 3644.58765765                                     
DISP = 2.06242371371                                     
Are these values OK (y/n) [y] ? 
Correct from atmospheric extinction (y/n) [y] ? 
1 - Curve from file
2 - Semi-analytical function
Option (1/2) [2] ? 1
Extinction curve file name [/files/extlp.dat] ? extlp.dat
No. points read:    201
Mean air mass [-999.000000] ? 1.20
Interpolating function...   ...OK
Correct from interstellar extinction (y/n) [y] ? n
Output file name? b2_1.uwssa
Output error file name [b2_1e.uwssa] ? 


```

<span style="color:darkblue">The final images were then converted again to FITS format:</span>

```
$ R5-writefits: b1_2.uwssa -> b1_2_uwssa.fits

$ R5-writefits: b2_1.uwssa -> b2_1_uwssa.fits

```

<span style="color:darkblue">We will use Python scripts for the further steps (see `2_Calibrating_OSIRIS_spectrum.ipynb`) such as the OBs combination, 1D spectrum extraction, absolute flux calibration + telluric correction, flux correction accounting for light losses in the slit, galactic extinction correction, and S/N estimation.</span>

